In [41]:
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome()
url = 'https://www.ratecity.com.au/savings-accounts'
driver.get(url)

time.sleep(2.5)

driver.execute_script("window.scrollTo(0, 1080)")

time.sleep(2.5)

driver.execute_script("window.scrollTo(0, 2080)")

get_today = datetime.datetime.now()
today = get_today.strftime('%d/%m/%Y')
affiliate = 'RateCity'
rank = 1

results = [['Date', 'Affiliate', 'Position', 'Provider', 'Product', 'Maximum Rate', 'Standard Rate', 'Max Rate Conditions', 'Savings Details']]

load_more_button = f'//*[@id="__next"]/div/main/div[3]/div/div[2]/div[4]/div[1]/div/button'
load_more_clicks = 2

for i in range(1, load_more_clicks):
    driver.execute_script('arguments[0].click();', WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, load_more_button))))
    time.sleep(2.5)
    
read_more = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="rc-ratetable"]/div/div//span[contains(text()," ...read more")]')))
for i in range(1, len(read_more) + 1):
    read_more_button = f'//*[@id="rc-ratetable"]/div/div//span[contains(text()," ...read more")]'
    driver.execute_script('arguments[0].click();', WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, read_more_button))))
    time.sleep(1.5)
    
savings = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="rc-ratetable"]/div/div')))
    
for i in range(1, len(savings) + 1) :
    savings_listing = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[2]/div[1]/div/div/div[2]/div/img'))).get_attribute('alt')      
    savings_product = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[2]/div[2]/div[1]/div/div[2]/p'))).text
    savings_max_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[3]/div[2]/div[1]/div[2]/div[1]/div/div[1]/p[1]'))).text.replace('*','')
    savings_standard_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[3]/div[2]/div[1]/div[2]/div[2]/div/div[1]/p[1]'))).text.replace('*','')
    
    try :
        savings_conditions = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[2]/div[2]/div[4]/p'))).text.replace(' ...read less','')
    except :
        savings_conditions = 'No max rate conditions listed'
    
    savings_details = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="rc-ratetable"]/div/div[{i}]/div[2]/div[2]/div[4]'))).text.replace(' ...read less','').replace('\n',' ').replace('Special','Special: ')  
    
    if savings_details == '' :
        savings_entry = f'{today} | {affiliate} | {rank} | {savings_listing} | {savings_product} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | No listed offer or details'
    else:
        savings_entry = f'{today} | {affiliate} | {rank} | {savings_listing} | {savings_product} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | {savings_details}'
    
    rank = rank + 1
    results.append(savings_entry.split('|'))

driver.quit()

df = pd.DataFrame(results[1:], columns=results[0]).to_dict('list')
df2 = pd.DataFrame(df)
    
print(df2)
df2.to_csv('EDBratecity.csv', index=False, encoding='utf-8')

          Date   Affiliate Position                                Provider  \
0  26/12/2022    RateCity        1                                   ubank    
1  26/12/2022    RateCity        2    Virgin Money (Australia) Pty Limited    
2  26/12/2022    RateCity        3                  Macquarie Bank Limited    
3  26/12/2022    RateCity        4                                ANZ Plus    
4  26/12/2022    RateCity        5    Virgin Money (Australia) Pty Limited    
5  26/12/2022    RateCity        6    Virgin Money (Australia) Pty Limited    
6  26/12/2022    RateCity        7                   Heritage Bank Limited    
7  26/12/2022    RateCity        8    Virgin Money (Australia) Pty Limited    
8  26/12/2022    RateCity        9                                     ING    
9  26/12/2022    RateCity       10                      Bank of Queensland    

                                    Product Maximum Rate Standard Rate  \
0                             Save Account        3.85% 